## 4. Объектно-ориентированное программирование

![OOP](https://raw.githubusercontent.com/amaargiru/pycore/main/pics/04_OOP.png)  

### Классы и объекты

Тут, конечно, было бы к месту кратенькое, минут на сорок, введеньице в тему классов и объектов, но в наш текущий формат такая мощная врезка не совсем укладывается. Попробую объяснить максимально просто, на доступных примерах из киновселенной «Чужих»:  
объект — это один конкретный ксеноморф;  
класс — это Королева ксеноморфов. Класс либо рожает ксеноморфа, либо может вступить в бой сам (@staticmethod);  
метапрограммирование — это такая Супер-Королева, размером с «Сулако», которая рожает других Королев;  
наследование — это ксеноморф из «Воскрешения», помните, миленький такой, взявший лучшее и от собственной генетической программы и от генов Рипли.  

Мы попробуем вернуться к теме объектов с чуть более серьезным настроением позже, в главе «Архитектура», но, вообще, в объектно-ориентированном программировании нет ничего особо сложного; просто до него лучше дойти, предварительно немного погрязнув в поддержке обычного процедурного подхода, когда зачастую стоит выбор — всё-таки попробовать подлечить этот кусок кода или уже усыпить его и переписать всё по новой? Когда-то давно, когда я писал относительно несложные программы на ассемблере для микроконтроллеров, то читая Страуструпа, слегка недоумевал — зачем всё это? Чтобы осознать потребность в обуви, надо походить босиком.

>__Что такое класс?__
>
> Класс - модель для создания объектов, описывающая их внутреннюю структуру и поведение.


### Магические методы

Специальные (называемые также magic или dunder) методы класса — перегрузка, позволяющая классам определять собственное поведение по отношению к операторам языка.  
Магические они потому, что почти никогда не вызываются явно, их вызывают встроенные функции или синтаксические конструкции. Например, функция len() вызывает метод \_\_len\_\_() переданного объекта. Метод \_\_add\_\_(self, other) вызывается автоматически при сложении оператором +.

Примеры магических методов:

\_\_init__: конструктор класса  
\_\_add__: сложение с другим объектом  
\_\_eq__: проверка на равенство с другим объектом  
\_\_cmp__: сравнение (больше, меньше, равно)  
\_\_iter__: используется при подстановке объекта в цикл  
\_\_new__: статический метод, вызываемый для создания экземпляра класса. [Официальная документация](https://docs.python.org/3/reference/datamodel.html#object.__new__) разъясняет назначение этого метода достаточно чётко — метод предназначен в основном для того, чтобы позволить подклассам неизменяемых типов (таких, как int, str или tuple) настраивать создание экземпляров либо для переопределения в пользовательских метаклассах для настройки создания классов.

>__В чем разница сравнения через == и через is?__
>
>== использует \_\_eq__ — магический метод проверки на равенство с другим объектом.
>
>is проверяет, указывают ли две переменные на один объект в памяти, «вручную» такое сравнение можно произвести при помощи функции id().

In [3]:
print(dir(int), "\n")


class A:  # An empty class
    ...


a = A()
print(dir(a), "\n")
print(repr(a), "\n")
print(str(a))

['__abs__', '__add__', '__and__', '__bool__', '__ceil__', '__class__', '__delattr__', '__dir__', '__divmod__', '__doc__', '__eq__', '__float__', '__floor__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getnewargs__', '__gt__', '__hash__', '__index__', '__init__', '__init_subclass__', '__int__', '__invert__', '__le__', '__lshift__', '__lt__', '__mod__', '__mul__', '__ne__', '__neg__', '__new__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rlshift__', '__rmod__', '__rmul__', '__ror__', '__round__', '__rpow__', '__rrshift__', '__rshift__', '__rsub__', '__rtruediv__', '__rxor__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__trunc__', '__xor__', 'as_integer_ratio', 'bit_count', 'bit_length', 'conjugate', 'denominator', 'from_bytes', 'imag', 'numerator', 'real', 'to_bytes'] 

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__

>__Что такое магические методы и для чего они нужны?__
>
>Специальные (называемые также magic или dunder) методы класса — перегрузка, позволяющая классам определять собственное поведение по отношению к операторам языка.  
>Магические они потому, что почти никогда не вызываются явно, их вызывают встроенные функции или синтаксические конструкции.
>
>Примеры магических методов:
>
>\_\_add__: сложение с другим объектом  
>\_\_cmp__: сравнение (больше, меньше, равно)  
>\_\_iter__: используется при подстановке объекта в цикл 

Особенностью метода \_\_init\_\_ является то, что он не должен ничего возвращать. При попытке возврата данных будет сгенерировано исключение.  
\_\_repr__ (representation) возвращает более-менее машино-читаемое представление объекта, полезное для отладки.  
*Иногда* \_\_repr может содержать достаточно информации для восстановления объекта.  
\_\_str\_\_ возвращает человеко-читаемое сообщение. Если \_\_str\_\_ не определён, то str использует repr.  

In [4]:
class Person:  # A simple class with init, repr and str methods
    def __init__(self, name: str):
        self.name: str = name

    def __repr__(self):
        return f"Person '{self.name}'"

    def __str__(self):
        return f"{self.name}"

    def say_hi(self):
        print("Hi, my name is", self.name)


p = Person("Charlie")
p.say_hi()
print(repr(p))
print(str(p))

Hi, my name is Charlie
Person 'Charlie'
Charlie


>__Как создается объект в Python? Какая разница между new и init?__
>
>Для создания объекта применяется специальная функция — конструктор, которая называется по имени класса и возвращает объект класса.
>
>Метод \_\_new__ используется, когда нужно управлять процессом создания нового экземпляра, а \_\_init__ — для контроля его инициализация. Поэтому \_\_new__ возвращает новый экземпляр класса, а \_\_init__ — ничего.

### @property

Декоратор [@property](https://docs.python.org/3/library/functions.html?highlight=property#property) используется для определения методов, доступных как поля. Таким образом операции чтения/записи поля можно обрамить дополнительной логикой, например, проверкой допустимых значений входного аргумента или пересчетом внутренних переменных объекта.

In [5]:
import math

class Circle:
    def __init__(self, radius, max_radius):
        self._radius = radius
        self.max_radius = max_radius

    @property
    def radius(self):
        return self._radius

    @radius.setter
    def radius(self, value):
        if value <= self.max_radius:
            self._radius = value
        else:
            raise ValueError

    @property
    def area(self):
        return 2 * self.radius * math.pi


circle = Circle(10, 100)
circle.radius = 20  # OK
# circle.radius = 101  # Raises ValueError
print(circle.area)

125.66370614359172


>__Что делает декоратор @property?__
>
>Декоратор [@property](https://docs.python.org/3/library/functions.html?highlight=property#property) используется для определения методов, доступных как поля. Таким образом операции чтения/записи поля можно обрамить дополнительной логикой, например, проверкой допустимых значений входного аргумента.

### @staticmethod

Обычный метод (т. е. не помеченный декораторами @staticmethod или @classmethod) имеет доступ к свойствам конкретного экземпляра класса.  

@staticmethod — метод, принадлежащий классу, а не экземпляру класса. Можно вызывать без создания экземпляра, т. к. метод не имеет доступа к свойствам экземпляра. При помощи @staticmethod помечают функционал, логически связанный с классом, но не требующий доступа к свойствам экземпляра.

### @classmethod, cls, self  

Если метод не должен иметь доступа к свойствам конкретного экземпляра класса (также, как @staticmethod), но должен иметь доступ к другим методам и переменным класса, то следует использовать @classmethod.

In [4]:
class B:
    def foo(self, x):
        print(f"Run foo({self}, {x})")

    @classmethod
    def class_foo(cls, x):
        print(f"Run class_foo({cls}, {x})")

    @staticmethod
    def static_foo(x):
        print(f"Run static_foo({x})")


b = B()
b.foo(1)
b.class_foo(1)
b.static_foo(1)

Run foo(<__main__.B object at 0x0000028571EA5850>, 1)
Run class_foo(<class '__main__.B'>, 1)
Run static_foo(1)


У @classmethod первым параметром должен быть cls (класс), а у обычного метода — self (экземпляр класса).  
Для @staticmethod не требуется ни cls, ни self.

### \_\_dict__

Каждый класс и каждый объект имеет атрибут \_\_dict__. Это системный, определённый интерпретатором атрибут, его не нужно создавать вручную. \_\_dict__ — словарь, который хранит пользовательские атрибуты; в нём ключом является _имя атрибута_, значением, соответственно, _значение атрибута_.

In [10]:
class Supercriminal:
    publisher = 'DC Comics'

Riddler = Supercriminal()
print(Supercriminal.__dict__)
print(Riddler.__dict__)

Riddler.name = 'Edward Nygma'
print(Riddler.__dict__)  # Values from object __dict__

print(Riddler.publisher)  # Value from class __dict__

{'__module__': '__main__', 'publisher': 'DC Comics', '__dict__': <attribute '__dict__' of 'Supercriminal' objects>, '__weakref__': <attribute '__weakref__' of 'Supercriminal' objects>, '__doc__': None}
{}
{'name': 'Edward Nygma'}
DC Comics


Каждый раз при запросе пользовательского атрибута Python последовательно обыскивает сам объект, класс объекта и классы, от которых унаследован класс объекта.

>__Для чего нужен атрибут dict?__
>
>Все классы и объекты в Python имеют атрибут \_\_dict__. Это определённый интерпретатором атрибут, его не нужно создавать вручную. \_\_dict__ — словарь, который хранит пользовательские атрибуты; в нём ключом является _имя атрибута_, значением, соответственно, _значение атрибута_.


### \_\_slots\_\_

Если вы припомните разницу между списком и кортежем, а также между множеством и иимутабельным множеством, то заметите, что создатели Python пытаются предоставлять разработчикам выбор между удобством и скоростью. К списку таких же особенностей языка, заточенных на увеличение производительности и уменьшение занимаемой памяти, относится и \_\_slots\_\_.

Вот [официальная документация](https://docs.python.org/3/reference/datamodel.html?highlight=slots#object.__slots__) по \_\_slots\_\_, а вот [дополнительные разъяснения](https://stackoverflow.com/questions/472000/usage-of-slots/28059785#28059785) от одного из разработчиков официальной документации. При выборе «slots или не slots» помните про существование [PEP 412 – Key-Sharing Dictionary](https://peps.python.org/pep-0412/), который внёс некоторый раздрай в некогда однозначное отношение к \_\_slots\_\_.

\_\_dict\_\_, рассмотренный чуть выше — изменяемая структура, и вы можете на лету добавлять и удалять поля из класса, что удобно, но порой медленно. Вы можете разменять удобство на скорость и размер занимаемой памяти, создав \_\_slots\_\_ — жестко заданный список предопределенных атрибутов, резервирующий память, создание которого _запрещает_ дальнейшее создание \_\_dict\_\_ и \_\_weakref\_\_. Слоты можно использовать, когда у класса может быть очень много полей, например, в ORM, либо когда критична производительность.

In [9]:
class Clan:
    __slots__ = ["first", "second"]

clan = Clan()
clan.first = "Joker"
clan.second = "Lex Luthor"
# clan.third = "Green Goblin"  # Raises AttributeError
# print(clan.__dict__)  # Raises AttributeError

Слоты используются, скажем, в библиотеках requests (например, \_\_slots\_\_ = ["url", "netloc", "simple_url", "pypi_url", "file_storage_domain"]) или ORM peewee (\_\_slots\_\_ = ('stack', '_sql', '_values', 'alias_manager', 'state')).

Наследование \_\_slots\_\_ имеет определенную специфику и будет рассмотрено ниже.

Чтобы было понятно, о каком приросте производительности и снижении потребления памяти идёт речь, сделаем простое сравнение:

In [19]:
import timeit
import pympler.asizeof  # В нашем случае sys.getsizeof — не лучший вариант, берем стороннее решение


class NotSlotted:
    pass


class Slotted:
    __slots__ = 'foo'


not_slotted = NotSlotted()
slotted = Slotted()


def get_set_delete_fn(obj):
    def get_set_delete():
        obj.foo = "Never Ending Song of Love"
        del obj.foo

    return get_set_delete


ns = min(timeit.repeat(get_set_delete_fn(not_slotted)))
s = min((timeit.repeat(get_set_delete_fn(slotted))))

print(ns, s, f'{(ns - s) / s * 100} %')

print(pympler.asizeof.asizeof(not_slotted), 'bytes')
print(pympler.asizeof.asizeof(slotted), 'bytes')

0.10838449979200959 0.08712740009650588 24.39772066187959 %
280 bytes
40 bytes


>__Что такое slots?__
>
>Атрибут \_\_slots__ позволяет ввести ограничение, задавая неизменяемый список атрибутов, которыми будет обладать экземпляр класса. За счет такого ограничения можно повысить скорость работы при доступе к атрибутам и сэкономить место в памяти.

На всякий случай напоминаю еще раз — прогоняйте все непонятные примеры кода в IDE, их можно и нужно анализировать, корректировать и видоизменять. Попробуйте, например, самостоятельно посмотреть потребление памяти объектов с \_\_dict\_\_ и \_\_slots\_\_. А заодно на практике испытайте давно напрашивающийся, и наконец появившийся в Python 3.10 [симбиоз](https://docs.python.org/3/library/dataclasses.html#module-contents) между \_\_slots\_\_ и dataclass.

### Утиная типизация

[Утиная типизация](https://en.wikipedia.org/wiki/Duck_typing) (duck types) — постулирование реализации интерфейса классом не через явное объявление, а через реализацию методов интерфейса. Так, каждый класс, реализующий методы \_\_next\_\_() и \_\_iter\_\_(), автоматически становится итератором, несмотря на отсутствие явного объявления (что-нибудь вроде @iterator) или, скажем, наследования от класса Iterator.

### Iterator

Итератор — класс, реализующий методы \_\_next\_\_() и \_\_iter\_\_().  
Метод \_\_next\_\_() должен возвращать следующее значение итератора или выкидывать исключение StopIteration, чтобы сигнализировать о том, что итератор исчерпал доступные значения.
Метод \_\_iter\_\_() должен возвращать "self".

In [8]:
class LimitCounter:
    def __init__(self, max_value: int):
        self.count = 0
        self.max_value = max_value

    def __next__(self):
        self.count += 1

        if self.count <= self.max_value:
            return self.count
        else:
            raise StopIteration

    def __iter__(self):
        return self


limit_counter = LimitCounter(2)
print(next(limit_counter))
print(next(limit_counter))


# print(next(limit_counter))  # Raises StopIteration

1
2


### Comparable

Начиная с Python 3.4, для того, чтобы экземпляры метода можно было сравнивать между собой, достаточно определить методы \_\_lt\_\_ (меньше) и \_\_eq\_\_ (равно), а также задействовать декоратор @functools.total_ordering.

In [9]:
from functools import total_ordering

@total_ordering
class Person:
    def __init__(self, firstname: str, lastname: str):
        self.firstname: str = firstname
        self.lastname: str = lastname

    def _is_valid_operand(self, other):
        return hasattr(other, "lastname") and hasattr(other, "firstname")

    def __eq__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return (self.lastname, self.firstname) == (other.lastname, other.firstname)

    def __lt__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return (self.lastname, self.firstname) < (other.lastname, other.firstname)


Finn = Person("Finn", "the Human")
Jake = Person("Jake", "the Dog")

print(Finn != Jake)

True


### Hashable

Хэшируемые объекты должны реализовывать методы \_\_hash\_\_() и \_\_eq\_\_(). Хеш объекта должен быть неизменен в течении всего жизненного цикла. Хешируемые объекты можно использовать как ключи в словарях и как элементы множеств, так как эти структуры используют хэш-таблицу для внутреннего представления данных.

Хэшируемые объекты, которые сравниваются между собой, должны иметь одинаковое хэш-значение, то есть стандартная hash(), возвращающая `'id(self)'`, не подойдет. Именно поэтому Python автоматически делает классы нехэшируемыми, если вы реализуете только eq().

In [13]:
class Hero:
    def __init__(self, name: str, level: int):
        self.name: str = name
        self.level: int = level

    def _is_valid_operand(self, other):
        return hasattr(other, "name") and hasattr(other, "level")

    def __eq__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return (self.name, self.level) == (other.name, other.level)

    def __hash__(self):
        return hash((self.name, self.level))


Finn = Hero("Finn the Human", 10_000)
Jake = Hero("Jake the Dog", 10_000)

print(hash(Finn))
print(hash(Jake))

-8707075988359731747
-2276052447712954388


### Sortable

Для возможности применения к последовательностям объектов таких методов как sort() или max() необходимо, как и в случае Comparable, определить методы \_\_lt__ (меньше) и \_\_eq__ (равно), а также задействовать декоратор @functools.total_ordering.

Для более предсказумого поведения объекта в условиях различного контекста (и, иногда, для оптимизации производительности) вы можете определить полное множество функций сравнения (\_\_lt()\_\_, \_\_gt()\_\_, \_\_le\_\_() и \_\_ge\_\_()).

Для примера создадим класс студентов, которых можно будет сортировать не по имени, а по среднему баллу:

In [14]:
from functools import total_ordering
from statistics import mean


@total_ordering
class Student:
    def __init__(self, name: str, grades: list[int]):
        self.name: str = name
        self.grades: list[int] = grades

    def _is_valid_operand(self, other):
        return hasattr(other, "name") and hasattr(other, "grades")

    def __eq__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return mean(self.grades) == mean(other.grades)

    def __lt__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return mean(self.grades) < mean(other.grades)

    # определим str для человеко-читаемой репрезентации объекта
    def __str__(self):
        return self.name + " " + str(mean(self.grades))


Melissa = Student("Melissa Andrew", [4, 3, 4, 5, 4])
Peter = Student("Peter Shining Jr.", [3, 3, 4, 5, 3])
Joe = Student("Just Joe", [5, 5, 4, 5, 5])

print([str(stud) for stud in sorted([Peter, Melissa, Joe], reverse=True)])

['Just Joe 4.8', 'Melissa Andrew 4', 'Peter Shining Jr. 3.6']


### Callable

Для возможности вызова объекта в качестве функции необходимо реализовать метод \_\_call\_\_. Типы, поддерживающие возможность их вызова в качестве функции, могут принимать набор аргументов.


In [3]:
class Counter:
    def __init__(self):
        self.i = 0
    def __call__(self):
        self.i += 1
        return self.i
 
counter = Counter()

print(counter())
print(counter())
print(counter())

1
2
3


@classmethod нельзя вызывать в качестве функции:

In [7]:
class Check():
    @classmethod 
    def class_method(cls):
        pass 

    @staticmethod
    def static_method():
        pass

    def instance_method(self):
        pass 

for attr, val in vars(Check).items():
    if not attr.startswith("__"):
        print (attr, f"{'is' if callable(val) else 'is NOT'} callable")

class_method is NOT callable
static_method is callable
instance_method is callable


### Контекстный менеджер

Код, размещенный внутри оператора with выполняется с особенностью: как до, так и после срабатывают события входа в блок with и выхода из него. Объект, который определяет логику событий, называется контекстным менеджером.

На уровне класса события определены методами \_\_enter\_\_ и \_\_exit\_\_:  
\_\_enter\_\_ срабатывает в тот момент, когда ход исполнения программы переходит внутрь with. Метод может вернуть значение, оно будет доступно расположенному внутри блока with коду;  
\_\_exit\_\_ срабатывает в момент выхода блока, в т.ч. и в случае исключения. В этом случае в метод будет передана тройка значений (exc_class, exc_instance, traceback).

Самый распространённый контекстный менеджер — класс, порожденный функцией open. Он гарантирует, что файл будет закрыт даже в том случае, если внутри блока возникнет ошибка.

Желательно побыстрее выходить из контекстного менеджера, освобождая контекст и ресурсы.

```python
with open('file.txt') as f:
    data = f.read()

process_data(data)
```

В примере выше мы вышли из блока with сразу же после прочтения файла. Обработка данных происходит в основном блоке программы.

>__Чем контекстный менеджер отличается от блока try-finally?__
>
>В целом, эти две конструкции весьма близки. Официальная [документация](https://docs.python.org/3/reference/compound_stmts.html#the-with-statement) даже рекомендует использовать with для удобного обёртывания try-finally. Есть [мнение](https://stackoverflow.com/questions/26096435/is-python-with-statement-exactly-equivalent-to-a-try-except-finally-bloc), что контекстный менеджер позволяет более гибко обрабатывать ошибки.

Контекстные менеджеры можно использовать для временной замены параметров, переменных окружения, транзакций БД.

Напишем свой контекстный менеджер для подключения к БД SQLite:

In [ ]:
import sqlite3


class db_conn:

    def __init__(self, db_name):
        self.db_name = db_name

    # Открываем подключение к БД
    def __enter__(self):
        self.conn = sqlite3.connect(self.db_name)
        return self.conn

    # Закрываем подключение к БД
    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.conn.close()
        if exc_value:
            raise


if __name__ == "__main__":
    db = "test_context_connect.db"

    with db_conn(db) as conn:
        cursor = conn.cursor()

>__Что такое контекстный менеджер?__
>
>Контекстный менеджер — механизм, обеспечивающий безопасное выполнение кода, связанного с управлением внешними ресурсами.
>
>Контекстный менеджер определяется методами \_\_enter\_\_ и \_\_exit\_\_. \_\_enter\_\_ срабатывает в момент перехода программы внутрь with. Метод может вернуть значение, оно будет доступно расположенному внутри блока with коду. \_\_exit\_\_ срабатывает в момент выхода из блока, в т.ч. и в случае исключения. В этом случае в метод будет передана тройка значений (имена аргументов на усмотрение разработчика) — exception_type (тип исключения), exception_instance (объект исключения), traceback (объект, содержащий информацию о последовательности вызовов, которые предшествовали исключению).

### Контекстный менеджер на базе contextlib

Перепишем наш контекстный менеджер для подключения к БД SQLite при помощи [contextlib](https://docs.python.org/3/library/contextlib.html):

In [ ]:
import sqlite3
from contextlib import contextmanager


# Схема конструирования следующая: всё, что написано до оператора yield — вызывается в рамках функции __enter__, всё что после – в рамках __exit__.
@contextmanager
def db_conn(db_name):
    # Открываем подключение к БД
    conn = sqlite3.connect(db_name)

    yield conn

    # Закрываем подключение к БД
    conn.close()


if __name__ == "__main__":
    db = "test_contextlib_connect.db"

    with db_conn(db) as conn:
        cursor = conn.cursor()

### Утиная типизация итерируемых объектов

### Iterable

[Iterable](https://docs.python.org/3/library/collections.abc.html#collections.abc.Iterable) — объект, который для предоставления возможности поочерёдного прохода по всем своим элементам должен реализовывать метод \_\_iter\_\_(), возвращающий итератор. У каждого объекта с методом \_\_iter\_\_() автоматически начинает работать метод \_\_contains\_\_().

In [15]:
class MyIterable:
    def __init__(self, *args):
        self.a = list(args)

    def __iter__(self):
        return iter(self.a)


mi = MyIterable(1, 2, 3, 4)
print([el for el in mi])
print(1 in mi)  # __contains__()

[1, 2, 3, 4]
True


### Collection

[Collection](https://docs.python.org/3/library/collections.abc.html#collections.abc.Collection) — объект, предоставляющий возможность поочерёдного прохода по всем своим элементам и обладающий конечным размером.  
В дополнение к iter() должен быть реализован метод len(), возвращающий размер коллекции.

In [16]:
class MyCollection:
    def __init__(self, *args):
        self.a = list(args)

    def __iter__(self):
        return iter(self.a)

    def __len__(self):
        return len(self.a)


mc = MyCollection(1, 2, 3, 4)
print([el for el in mc])
print(1 in mc)
print(len(mc))

[1, 2, 3, 4]
True
4


### Sequence

Требует методы len() and getitem(). getitem() должен отдавать элемент с требуемым индексом или вызывать исключение IndexError.  
Автоматически будут порождены методы iter(), reversed() и contains().

In [8]:
class MySequence:
    def __init__(self, a):
        self.a = a
    def __len__(self):
        return len(self.a)
    def __getitem__(self, i):
        return self.a[i]

### ABC Sequence

Коллекция Sequence из [Abstract Base Classes for Containers](https://docs.python.org/3/library/collections.abc.html) предоставляет расширенный интерфейс по сравнению с обычной Sequence.  
Всё так же требуя \_\_getitem\_\_ и \_\_len\_\_, предоставляет \_\_contains__, \_\_iter\_\_, \_\_reversed\_\_, index и count.

In [9]:
from collections import abc

class MyAbcSequence(abc.Sequence):
    def __init__(self, a):
        self.a = a
    def __len__(self):
        return len(self.a)
    def __getitem__(self, i):
        return self.a[i]

### Таблица требуемых и доступных методов:

```text
+------------+------------+------------+------------+--------------+
|            |  Iterable  | Collection |  Sequence  | ABC Sequence |
+------------+------------+------------+------------+--------------+
| iter()     |   нужен    |   нужен    |     +      |      +       |
| contains() |     +      |     +      |     +      |      +       |
| len()      |            |   нужен    |   нужен    |    нужен     |
| getitem()  |            |            |   нужен    |    нужен     |
| reversed() |            |            |     +      |      +       |
| index()    |            |            |            |      +       |
| count()    |            |            |            |      +       |
+------------+------------+------------+------------+--------------+
```

И вообще, потщательнее присмотритесь с collections.abc, там есть множество заготовок, которые помогут вам сэкономить немало времени. Например, если к упомянутым \_\_getitem\_\_ и \_\_len\_\_ добавить \_\_setitem\_\_, \_\_delitem\_\_ и insert, то в ответ вы получите коллекцию MutableSequence, которая, кроме возможностей Sequence, имеет еще методы append, reverse, extend, pop, remove и \_\_iadd\_\_.

### Копирование объектов

В Python оператор присваивания (=) не копирует объекты. Вместо этого он создает связь между существующим объектом и именем целевой переменной. [Вот тут](https://nedbatchelder.com/text/names1.html) есть хорошее объяснение происходящего.

In [5]:
nums = [1, 2, 3]
other = nums
nums.append(4)  # Изменятся и nums, и other

print(other)

[1, 2, 3, 4]


Чтобы создать копии объекта в Python, необходимо использовать модуль copy. Существует два способа создания копий объекта.  

Shallow Copy (поверхностная копия) – копирует сам объект, вложенные объекты не копируются, они доступны по тем же ссылкам.

Deep Copy – рекурсивно копирует все вложенные объекты.

In [17]:
from copy import copy, deepcopy


class A:
    def __init__(self, val: list):
        self.val = val

    def change_val(self, val: list):
        self.val = val


a = A(list("one"))

# Просто копирование ссылки на объект
b = a  # Assignment

# Создание нового объекта и копирование ссылок на объекты, найденные в изначальном объекте
c = copy(a)  # Shallow copy

# Создание нового объекта с последующим рекурсивным копированием содержащихся внутри объектов
d = deepcopy(a)  # Deep Copy

b.change_val(list("two"))
c.change_val(list("three"))
d.change_val(list("four"))

print(a.val, b.val, c.val, d.val)
print(id(a), id(b), id(c), id(d))
print(id(a.val[1]), id(c.val[1]))

['t', 'w', 'o'] ['t', 'w', 'o'] ['t', 'h', 'r', 'e', 'e'] ['f', 'o', 'u', 'r']
1795295519472 1795295519472 1793149281968 1793149273808
1795217224688 1795217321264


>__Что такое поверхностная копия? Что такое глубокая копия?__  
>
> При поверхностном копировании вложенные объекты не копируются, копируются только ссылки на них. При использовании глубокого копирования рекурсивно копируются все вложенные объекты.

В Python 3.13 появилась возможность копировать и одновременно изменять копируемый объект при помощи copy.replace:

In [9]:
from collections import namedtuple

Point = namedtuple('Point', ['x', 'y'])
p1 = Point(20, 30)
print(f"Original Point: {p1}")

p2 = copy.replace(p1, x=25)
print(f"Modified Point: {p2}")

Original Point: Point(x=20, y=30)
Modified Point: Point(x=25, y=30)


Такой же финт можно провернуть с dataclass:

In [14]:
from dataclasses import dataclass

@dataclass
class Employee:
  name: str 
  deparment: str

user1 = Employee('Alice', 'HR')

user2 = copy.replace(user1, name='Bob')

print(user1)
print(user2)

Employee(name='Alice', deparment='HR')
Employee(name='Bob', deparment='HR')


### Наследование

In [1]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age


class Employee(Person):
    def __init__(self, name, age, staff_num, email):
        super().__init__(name, age)
        self.staff_num = staff_num
        self.email = email

### Множественное наследование (MRO)

При множественном наследовании порядок разрешения методов (method resolution order, MRO) позволяет Питону выяснить, из какого родительского класса нужно вызывать метод, если он не обнаружен непосредственно в классе-потомке.

In [3]:
class PrivateStaffData:
    def __init__(self, private_email):
        self.private_email = private_email


class PublicStaffData:
    def __init__(self, work_email):
        self.work_email = work_email


class StaffData(PrivateStaffData, PublicStaffData):
    def __init__(self, private_email, work_email):
        super().__init__()

print(StaffData.mro())

[<class '__main__.StaffData'>, <class '__main__.PrivateStaffData'>, <class '__main__.PublicStaffData'>, <class 'object'>]


MRO строит иерархию наследования таким образом, чтобы более специфичные методы класса-потомка перекрывали менее специфичные методы класса-предка. MRO строит упорядоченный список классов, в которых будет производиться поиск метода слева направо (линеаризация класса).

Для решения проблемы ромбовидной структуры (которая неявно присутствует даже в простейшем случае, так как все классы наследуются от object) линеаризация должна быть монотонной. Монотонность — свойство, которое требует соблюдения в линеаризации класса-потомка того же порядка следования классов-прародителей, что и в линеаризации класса-родителя.  Линеаризация по сути является [топологической сортировкой](https://en.wikipedia.org/wiki/Topological_sorting). В ранних версиях Python использовался алгоритм DLR, сейчас в ходу [C3-линеаризация](https://en.wikipedia.org/wiki/C3_linearization).  

Если после удовлетворения свойства монотонности остаётся больше одного варианта линеаризации, то применяется порядок локального старшинства (local precedence ordering), т. е. порядок соблюдения для классов-родителей в линеаризации класса-потомка того же порядка, что и при его объявлении. Например, если класс объявлен как D(A, B, C), то в линеаризации D класс A должен стоять раньше B, а класс B — раньше C.  

Если разрешение всех конфликтов при линеаризации невозможно, то остается три пути:  
1 — переменой мест классов-предков в объявлении класса-потомка (но это помогает далеко не всегда);  
2 — пересмотр иерархии наследования;  
3 — определение своей собственной линеаризации через метаклассы при помощи метода mro(cls). Но при данном подходе надо быть готовым к тому, что будет использован менее специфичный метод класса-родителя вместо более специфичного метода класса-потомка.  

При задании своей собственной линеаризации Python отключает встроенные проверки.  

>__Что такое MRO?__
>
>MRO (method resolution order, порядок разрешения методов) — механизм, позволяющий при множественном наследовании определить родительский класс. MRO строит упорядоченный список классов, в которых будет производиться поиск метода слева направо (производит линеаризацию класса).

### Mixin

Миксин — класс, предназначенный для добавления определенной функциональности к другим классам через наследование. Он не является самостоятельным классом, а служит скорее для сведения однотипного кода в одно место, позволяя избежать дублирования.

In [17]:
import json

# Миксин для преобразования объекта в JSON
class JsonMixin:
    def to_json(self):
        return json.dumps(self.__dict__)

# Миксин для преобразования объекта в текст
class TextMixin:
    def to_text(self):
        return f"{self.__dict__}"

# Класс Animal использует оба миксина
class Animal(JsonMixin, TextMixin):
    def __init__(self, name, species):
        self.name = name
        self.species = species

# Создаем объект и используем методы миксинов
dog = Animal("Buddy", "Dog")
print(dog.to_json())
print(dog.to_text())

{"name": "Buddy", "species": "Dog"}
{'name': 'Buddy', 'species': 'Dog'}


### @abstractmethod

Абстрактный класс в Python — аналог интерфейса в других языках (например, в C#) — класс, содержащий только сигнатуры методов, без реализации. Реализация методов переложена на классы-потомки. Задача абстрактного класса соответствует задаче интерфейса — *обязать* классы-потомки реализовывать *все* методы, заложенные в классе-родителе.

In [ ]:
import abc


class AbstractClass(metaclass=abc.ABCMeta):

    @abc.abstractmethod
    def return_anything(self):
        return


class ConcreteClass(AbstractClass):

    def return_anything(self):
        return 42


c = ConcreteClass()
print(c.return_anything())

42


Если не специфицировать return_anything() в ConcreteClass, при попытке вызвать c.return_anything() будет выброшено исключение _TypeError: Can't instantiate abstract class ConcreteClass with abstract method return_anything._


### Наследование классов со \_\_slots\_\_

При одиночном наследовании \_\_slots\_\_ нормально наследуется, но это не предотвращает создание \_\_dict\_\_:

In [16]:
class SlotsClass:
  __slots__ = 'foo', 'bar'

  
class ChildSlotsClass(SlotsClass):
  ...


obj = ChildSlotsClass()
print(obj.__slots__)

obj.something_new = "underwater stones"
print(obj.__dict__)

('foo', 'bar')
{'something_new': 'underwater stones'}


Для ограничения дочернего класса слотами нужно в нём снова присвоить значение атрибуту \_\_slots\_\_, родительские поля дублировать не нужно.

In [17]:
class SlotsClass:
  __slots__ = 'foo', 'bar'

  
class ChildSlotsClass(SlotsClass):
  __slots__ = 'baz'


obj = ChildSlotsClass()
# obj.something_new = "underwater stones"  # Raises AttributeError: 'ChildSlotsClass' object has no attribute 'something_new'

Множественное же наследование классов с _непустыми_ \_\_slots\_\_ невозможно.

### Метапрограммирование

Что такое класс? Это, в принципе, просто кусок кода, описывающий, как создать объект. Но в Python класс — это нечто большее, классы также являются объектами; как только используется ключевое слово class, Python исполняет команду и создаёт объект:

In [10]:
class A:
    ...

В памяти будет создан объект с именем A.

Классы, как и другие объекты, можно создавать на ходу:

In [11]:
def custom_class(name):
    if name == "foo":
        class Foo:
            ...

        return Foo  # Возвращает именно класс, а не экземпляр
    else:
        class Bar:
            ...

        return Bar


MyClass = custom_class("foo")
print(MyClass)  # Функция возвращает класс, а не экземпляр
print(my_class := MyClass())  # Можно создать экземпляр класса

<class '__main__.custom_class.<locals>.Foo'>
<__main__.custom_class.<locals>.Foo object at 0x000001F0ECF97610>


Но это не очень удобно, так как нам до сих пор приходится писать весь код класса.

Основная цель метаклассов — автоматически изменять класс в момент создания, генерируя классы в соответствии с текущим контекстом.  
Сами по себе метаклассы достаточно просты и работают примерно следующим образом:  
перехватывают создание класса,  
изменяют класс,  
возвращают модифицированный класс.

Но обычно логику работы метаклассов насыщают вещами вроде [интроспекции](https://en.wikipedia.org/wiki/Type_introspection#Python) или манипуляцией наследованием, поэтому конечный код выглядит достаточно громоздко.

Здесь неплохо было бы добавить еще пару страниц про ньюансы создания и работы метаклассов, но позвольте переадресовать вас на вот эту [прекрасную статью](https://habr.com/ru/post/145835/).

При помощи метаклассов хорошо решаются задачи, например, генерации классов для ORM. Скажем, для
```python
class Person(models.Model):
    name = models.CharField(max_length=30)
    age = models.IntegerField()
```
код
```python
keanu = Person(name="Keanu Reeves", age=58)
print(keanu.age)
```
распечатает число, взятое из БД, потому что models.Model определяет \_\_metaclass\_\_, который сотворит некоторую магию и превратит класс Person, определённый достаточно простым выражением, в сложную привязку к базе данных.

>__Что такое метаклассы?__
>
>Классы, как и прочие объекты, можно создавать во время исполнения программы. Основная цель метаклассов — автоматически изменять класс в момент создания, генерируя классы в соответствии с текущим контекстом. Сами по себе метаклассы достаточно просты и работают примерно следующим образом:  
>перехватывают создание класса,  
>изменяют класс,  
>возвращают модифицированный класс.

Если вы всё еще ломаете голову, где бы вам использовать метапрограммирование в своём текущем проекте, чтобы потом можно было упомянуть об этом в резюме, то вот вам на всякий случай цитата из [Тима Питерса](https://en.wikipedia.org/wiki/Tim_Peters_(software_engineer)): «\[Metaclasses\] are deeper magic than 99% of users should ever worry about. If you wonder whether you need them, you don’t (the people who actually need them know with certainty that they need them, and don’t need an explanation about why)».